# Hybrid Recommendation System using H&M Dataset

This notebook demonstrates the process of building and evaluating a hybrid recommendation system using the [H&M Personalized Fashion Recommendations dataset](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations). The process includes loading datasets, preprocessing, splitting data temporally, and training multiple recommendation models. 

The key steps are as follows:

1. **Dataset Loading**:
    - Load the H&M transactions dataset, user tags, and item tags using utility functions from the `rtrec` library.

2. **Data Preprocessing**:
    - Sample a subset of the transactions dataset for efficient processing.
    - Inspect the dataset structure and information.
    - Split the transactions dataset into training and testing sets based on time, ensuring realistic evaluation.

3. **Model Initialization**:
    - Initialize multiple recommendation models, including:
      - `HybridSlimFM`: A hybrid model combining SLIM and Factorization Machines.
      - `SLIM`: Sparse Linear Methods for recommendation.
      - `LightFM`: A model combining collaborative and content-based filtering.

4. **Training**:
    - Train the models using the training dataset, incorporating user and item tags where applicable.

5. **Evaluation**:
    - Evaluate the models on the test dataset using metrics like recommendation accuracy and filtering interacted items.

This notebook provides a comprehensive workflow for experimenting with hybrid recommendation systems and comparing the performance of different models using the H&M dataset.

In [1]:
from rtrec.experiments.kaggle_datasets import load_hm_dataset

transactions = load_hm_dataset()
transactions

File already exists at datasets/kaggle/h-and-m/transactions_train.csv.


,user,item,price,sales_channel_id,tstamp,rating
0,9216525696054190872,663713001,0.050831,2,1.537402e+09,1.0
1,9216525696054190872,541518023,0.030492,2,1.537402e+09,1.0
2,888740269715967170,505221004,0.015237,2,1.537402e+09,1.0
3,888740269715967170,685687003,0.016932,2,1.537402e+09,1.0
4,888740269715967170,685687004,0.016932,2,1.537402e+09,1.0
...,...,...,...,...,...,...
31788319,4685485978980270934,929511001,0.059305,2,1.600733e+09,1.0
31788320,4685485978980270934,891322004,0.042356,2,1.600733e+09,1.0
31788321,3959348689921271969,918325001,0.043203,1,1.600733e+09,1.0
31788322,584031991477264143,833459002,0.006763,1,1.600733e+09,1.0


In [2]:
transactions = transactions.sample(frac=0.05, random_state=42)

In [3]:
transactions

,user,item,price,sales_channel_id,tstamp,rating
16558544,3915371767870068292,786586001,0.022017,1,1.568333e+09,1.0
6583409,5056261243377697188,658911001,0.028797,2,1.550880e+09,1.0
13976622,5222503288154568012,759326005,0.050831,2,1.563322e+09,1.0
10327778,1187699526868400021,737137004,0.027102,1,1.557965e+09,1.0
15285658,4727407853099057127,785931001,0.050831,2,1.565395e+09,1.0
...,...,...,...,...,...,...
12877254,99934464486908387,774167003,0.101678,2,1.561680e+09,1.0
14972391,4865465219017154622,767577011,0.025407,1,1.564790e+09,1.0
3439878,3837155778372452700,198518010,0.016932,2,1.544054e+09,1.0
15883711,5008351590642632763,465655010,0.016932,2,1.566605e+09,1.0


In [4]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1589416 entries, 16558544 to 14435344
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   user              1589416 non-null  int64  
 1   item              1589416 non-null  Int32  
 2   price             1589416 non-null  float32
 3   sales_channel_id  1589416 non-null  Int8   
 4   tstamp            1589416 non-null  float64
 5   rating            1589416 non-null  float64
dtypes: Int32(1), Int8(1), float32(1), float64(2), int64(1)
memory usage: 65.2 MB


In [5]:
transactions

,user,item,price,sales_channel_id,tstamp,rating
16558544,3915371767870068292,786586001,0.022017,1,1.568333e+09,1.0
6583409,5056261243377697188,658911001,0.028797,2,1.550880e+09,1.0
13976622,5222503288154568012,759326005,0.050831,2,1.563322e+09,1.0
10327778,1187699526868400021,737137004,0.027102,1,1.557965e+09,1.0
15285658,4727407853099057127,785931001,0.050831,2,1.565395e+09,1.0
...,...,...,...,...,...,...
12877254,99934464486908387,774167003,0.101678,2,1.561680e+09,1.0
14972391,4865465219017154622,767577011,0.025407,1,1.564790e+09,1.0
3439878,3837155778372452700,198518010,0.016932,2,1.544054e+09,1.0
15883711,5008351590642632763,465655010,0.016932,2,1.566605e+09,1.0


In [ ]:
from rtrec.experiments.kaggle_datasets import load_hm_user_tags
user_tags = load_hm_user_tags()
user_tags

In [ ]:
from rtrec.experiments.kaggle_datasets import load_hm_item_tags

item_tags = load_hm_item_tags()
item_tags

In [8]:
from rtrec.experiments.split import temporal_split

train_df, test_df = temporal_split(transactions, test_frac=0.2)

In [9]:
from rtrec.recommender import Recommender
from rtrec.models import HybridSlimFM

hybrid_model = HybridSlimFM(no_components=10, epochs=20, use_bias=True, nn_feature_selection=50, force_identify=True, decay_in_days=180)
hybrid_recommender = Recommender(hybrid_model)

/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from rtrec.models import SLIM

slim_model = SLIM(epochs=20, use_bias=True, nn_feature_selection=50, force_identify=True, decay_in_days=180)
slim_recommender = Recommender(slim_model)

In [11]:
from rtrec.recommender import Recommender
from rtrec.models import LightFM

fm_model = LightFM(no_components=10, loss="warp", epochs=20, use_bias=True, force_identify=True, decay_in_days=180)
fm_recommender = Recommender(fm_model)

In [12]:
train_user_tags = {int(user): user_tags[user] for user in train_df.user.unique() if user in user_tags}

In [13]:
train_item_tags = {int(item): item_tags[item] for item in train_df.item.unique() if item in item_tags}

In [14]:
hybrid_recommender.bulk_fit(train_df, user_tags=train_user_tags, item_tags=train_item_tags)

Fitting SLIMElastic in parallel:   0%|          | 0/681 [00:00<?, ?it/s]/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarn

Fit completed in 126.79 seconds
Throughput: 10028.51 samples/sec


In [15]:
test_user_tags = {int(user): user_tags[user] for user in test_df.user.unique() if user in user_tags}

In [16]:
len(test_df.user.unique())

209728

In [17]:
len(set(test_df.user.unique()) - set(train_df.user.unique()))

79392

In [18]:
hybrid_recommender.evaluate(test_df, test_user_tags, recommend_size=10, filter_interacted=True)

100%|██████████| 2098/2098 [58:11<00:00,  1.66s/it]    


{'precision': 0.0006699153188892107,
 'recall': 0.005002651376168514,
 'f1': 0.0011476094710875515,
 'ndcg': 0.0027577877027014606,
 'hit_rate': 0.00663716814159292,
 'mrr': 0.00245565987813089,
 'map': 0.0018575420030896475,
 'tp': 1405,
 'auc': 0.003837609506916904}

In [19]:
fm_recommender.bulk_fit(train_df, user_tags=train_user_tags, item_tags=train_item_tags)

Epoch: 100%|██████████| 20/20 [00:50<00:00,  2.50s/it]

Fit completed in 55.72 seconds
Throughput: 22818.09 samples/sec


In [20]:
fm_recommender.evaluate(test_df, test_user_tags, recommend_size=10, filter_interacted=True)

100%|██████████| 2098/2098 [16:02<00:00,  2.18it/s]


{'precision': 0.0006661008544400193,
 'recall': 0.004957311474049312,
 'f1': 0.001139784709161968,
 'ndcg': 0.0027348030098272346,
 'hit_rate': 0.006632400061031431,
 'mrr': 0.0024556163599352854,
 'map': 0.0018337039789164968,
 'tp': 1397,
 'auc': 0.003877906409995604}

In [21]:
slim_recommender.bulk_fit(train_df)

Fitting SLIMElastic in parallel:   0%|          | 0/681 [00:00<?, ?it/s]/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/myui/workspace/myui/rtrec/.venv/lib/python3.12/site-packages/lightfm/_lightfm_fast.py:9: UserWarn

Fit completed in 75.21 seconds
Throughput: 16906.85 samples/sec


In [22]:
slim_recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|██████████| 2098/2098 [04:49<00:00,  7.24it/s]


{'precision': 0.000411962160512661,
 'recall': 0.003301795507771821,
 'f1': 0.0007180679168625928,
 'ndcg': 0.001808729129026709,
 'hit_rate': 0.004110085444003662,
 'mrr': 0.0015615236787421703,
 'map': 0.0012383621307739685,
 'tp': 864,
 'auc': 0.002470958415980065}

### Evaluation Results Explanation

The evaluation results demonstrate that the **Hybrid model** outperforms both the **Collaborative Filtering (CF)** model (SLIM) and the **Content-Based (CB)** model (LightFM) in terms of **Mean Average Precision (MAP)** and **Normalized Discounted Cumulative Gain (NDCG)**. Here's a breakdown of the metrics and the reasons for the Hybrid model's superior performance:

---

#### **Metrics Explanation**
1. **Mean Average Precision (MAP)**:
    - MAP measures the precision of recommendations at different cutoff points, averaged across all users.
    - It emphasizes the relevance of the recommended items and rewards models that rank relevant items higher.

2. **Normalized Discounted Cumulative Gain (NDCG)**:
    - NDCG evaluates the ranking quality of recommendations by considering the position of relevant items in the ranked list.
    - Higher relevance scores for items appearing earlier in the list result in higher NDCG values.

---

#### **Performance Comparison**
- **SLIM (CF)**:
  - Performs well for users with many interactions (hot-users) because it leverages collaborative signals from user-item interaction patterns.
  - Struggles with cold-users who have limited or no interaction history.

- **LightFM (CB)**:
  - Excels in cold-user scenarios by utilizing user and item metadata (e.g., tags, attributes).
  - However, it may not fully capture collaborative signals, leading to suboptimal performance for hot-users.

- **Hybrid Model**:
  - Combines the strengths of both CF and CB approaches.
  - For hot-users, it leverages collaborative signals from SLIM.
  - For cold-users, it incorporates content-based signals from LightFM, ensuring robust performance across diverse user groups.

---

#### **Reason for Hybrid Model's Superiority**
- **Hot-User Performance**:
  - CF models like SLIM are well-suited for hot-users with rich interaction histories, as they rely on collaborative signals.
  - The Hybrid model incorporates these collaborative signals, ensuring competitive performance for hot-users.

- **Cold-User Performance**:
  - CB models like LightFM are essential for cold-users with sparse or no interaction history, as they rely on metadata.
  - The Hybrid model integrates these content-based signals, addressing the cold-start problem effectively.

- **Balanced Approach**:
  - By combining CF and CB signals, the Hybrid model achieves a balance, outperforming CF-only and CB-only models in both MAP and NDCG metrics.

---

### Conclusion
The Hybrid model's ability to adapt to both hot-user and cold-user scenarios explains its superior MAP and NDCG results. This adaptability ensures that the model provides high-quality recommendations across a diverse user base, making it a robust choice for real-world recommendation systems.